In [2]:
import torch
import torch.nn as nn
from d2l import torch as d2l

In [ ]:
def dropout_layer(x,dropout):
    assert 0<= dropout <= 1
    if dropout == 1: # 全部丢弃
        return torch.zeros_like(x)
    elif dropout == 0:
        return x # 不丢弃
    mask = (torch.randn(x.shape) > dropout).float()

    # 也相当于是一个bool数组，随机初始化一个形状和x一样的数组，然后大于dropout为1 小于为0
    return mask * x / (1.0 - dropout)
    # 乘法比bool索引要更快一点 x[mask]
    

In [4]:
import torch
import timeit

# 定义 dropout_layer 函数（稍微修改一下，使其能接受布尔索引版本）
def dropout_layer_multiply(x, dropout):
    """使用乘法进行 dropout"""
    assert 0 <= dropout <= 1
    if dropout == 1:
        return torch.zeros_like(x)
    elif dropout == 0:
        return x
    mask = (torch.randn(x.shape) > dropout).float()
    return mask * x / (1.0 - dropout)

def dropout_layer_bool_index(x, dropout):
    """使用布尔索引进行 dropout"""
    assert 0 <= dropout <= 1
    if dropout == 1:
        return torch.zeros_like(x)
    elif dropout == 0:
        return x
    mask = (torch.randn(x.shape) > dropout).float()
    # 注意：布尔索引会直接移除被 mask 为 0 的元素，
    # 为了模拟乘法操作的输出形状，我们需要重新构建一个相同形状的张量
    # 这是一个关键的差异，直接的布尔索引是改变形状的
    # 为了公平比较，我们这里假设目标是得到一个与输入 x 形状相同的张量
    # 实际应用中，如果 dropout != 0 and dropout != 1，
    # 乘法版本的输出形状和输入形状一致，而布尔索引的输出形状会变小。
    # 为了让测试更具可比性，这里我们模拟一种场景，即需要保持原形状。
    # 一个常见的实现是先创建全零张量，然后用布尔索引填充。
    # 但这样做会引入额外的操作，可能掩盖原始布尔索引的性能。
    #
    # 让我们采取一个更直接的比较，即比较生成 mask 后，
    # 如何应用这个 mask 来“丢弃”元素，并保持原形状。

    # 另一种更接近乘法操作的布尔索引模拟：
    # 创建一个与 x 形状相同的全零张量
    y = torch.zeros_like(x)
    # 使用布尔索引将对应位置的 x 的值赋给 y
    y[mask.bool()] = x[mask.bool()]
    # 然后进行归一化
    return y / (1.0 - dropout)

# -------------------------------------------------------------------------
# 准备测试数据
torch.manual_seed(42) # 为了结果的可复现性
data_size = (1000, 1000) # 可以根据需要调整张量的大小
x_tensor = torch.randn(*data_size)
dropout_rate = 0.5 # 0.5 表示丢弃 50% 的元素

# -------------------------------------------------------------------------
# 设置 timeit 的参数
number_of_runs = 100 # 执行测试的次数

# -------------------------------------------------------------------------
# 测试乘法版本
setup_multiply = """
import torch
from __main__ import dropout_layer_multiply
x = torch.randn({size}, device='cpu') # 在 CPU 上测试，以避免 GPU 调度开销
dropout = {dropout}
""".format(size=data_size, dropout=dropout_rate)

time_multiply = timeit.timeit(
    "dropout_layer_multiply(x, dropout)",
    setup=setup_multiply,
    number=number_of_runs
)

print(f"乘法版本的平均执行时间 ({number_of_runs} 次运行): {time_multiply / number_of_runs:.6f} 秒")

# -------------------------------------------------------------------------
# 测试布尔索引版本
setup_bool_index = """
import torch
from __main__ import dropout_layer_bool_index
x = torch.randn({size}, device='cpu') # 在 CPU 上测试
dropout = {dropout}
""".format(size=data_size, dropout=dropout_rate)

time_bool_index = timeit.timeit(
    "dropout_layer_bool_index(x, dropout)",
    setup=setup_bool_index,
    number=number_of_runs
)

print(f"布尔索引版本的平均执行时间 ({number_of_runs} 次运行): {time_bool_index / number_of_runs:.6f} 秒")

# -------------------------------------------------------------------------
# 验证结果是否一致（理论上应该非常接近，但由于随机性可能略有差异）
print("\n验证结果的一致性:")
result_multiply = dropout_layer_multiply(x_tensor, dropout_rate)
result_bool_index = dropout_layer_bool_index(x_tensor, dropout_rate)

# 检查形状
print(f"乘法结果形状: {result_multiply.shape}")
print(f"布尔索引结果形状: {result_bool_index.shape}")

# 检查数值是否接近
# 由于 mask 是随机生成的，每次运行 mask 都会不同，
# 所以我们不能直接比较两个函数调用结果的数值。
# 我们需要比较的是，对于**相同的随机 mask**，两种方法的结果是否一致。

# 重新生成一个相同的 mask
torch.manual_seed(42) # 重新设置种子以确保 mask 一致
x_test = torch.randn(*data_size)
mask_test = (torch.randn(x_test.shape) > dropout_rate).float()

# 重新实现一次乘法和布尔索引，使用相同的 mask
def dropout_layer_multiply_with_mask(x, mask, dropout):
    return mask * x / (1.0 - dropout)

def dropout_layer_bool_index_with_mask(x, mask, dropout):
    y = torch.zeros_like(x)
    y[mask.bool()] = x[mask.bool()]
    return y / (1.0 - dropout)

result_multiply_fixed_mask = dropout_layer_multiply_with_mask(x_test, mask_test, dropout_rate)
result_bool_index_fixed_mask = dropout_layer_bool_index_with_mask(x_test, mask_test, dropout_rate)

# 比较数值
print(f"数值是否接近 (对于相同 mask): {torch.allclose(result_multiply_fixed_mask, result_bool_index_fixed_mask)}")


乘法版本的平均执行时间 (100 次运行): 0.008449 秒
布尔索引版本的平均执行时间 (100 次运行): 0.019245 秒

验证结果的一致性:
乘法结果形状: torch.Size([1000, 1000])
布尔索引结果形状: torch.Size([1000, 1000])
数值是否接近 (对于相同 mask): True


y = torch.zeros_like(x)
使用布尔索引将对应位置的 x 的值赋给 y
y[mask.bool()] = x[mask.bool()]

对于这段代码，有bool索引的知识点，bool索引会打算原来tensor的形状，只返回一个1维数组，哪些元素符合条件
返回的是一个副本，不是引用  
但是当bool索引出现在 赋值符号（=）左侧时，是另外一种情况  
这部分是赋值目标的指定。
当布尔索引出现在赋值操作的左侧时，它并不创建一个新的张量。
相反，它标识了目标张量 y 中的哪些位置应该被更新。
mask.bool() 在这里的作用是告诉 PyTorch：“请找到 y 中所有在 mask.bool() 中对应值为 True 的位置。”
PyTorch 会为这些被 True 标记的位置准备好接收新值的空间。  
赋值操作的目的是将等号右侧的值，写入到等号左侧指定的位置。
 

In [5]:
# 定义超参数 或者说模型的参数
input_size = 28*28 
output_size = 10
hidden1_size = 256
hidden2_size = 256

# 定义每一层dropout的概率
dropout1 = 0.2
dropout2 = 0.5

In [ ]:
from operator import is_
from tarfile import is_tarfile
from turtle import forward


class Net(nn.Module):
    def __init__(self,input_size,output_size,hidden1_size,hidden2_size,is_training = True):
        super().__init__()
        self.is_training = is_training
        self.fc1 = nn.Linear(input_size,hidden1_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size,hidden2_size)
        self.fc3 = nn.Linear(hidden2_size,output_size)

    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)

        if self.is_tarining:
            out = dropout_layer(out,dropout1)
        
        out = self.fc2(out)
        out = self.relu(out)

        if self.is_training:
            out = dropout_layer(out ,dropout2)
        # 交叉熵损失函数会自动计算softmax  所以我们这不需要最后经过softmax
        return out

    

In [ ]:
net = Net(input_size,output_size,hidden1_size,hidden2_size)